In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as func
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import NaiveBayes, NaiveBayesModel
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

In [3]:
smoothing = float(1.0)

In [4]:
### Initialize streaming context
conf = SparkConf()\
                .setMaster("local[2]")\
                .setAppName("MobileAnalyticsNaiveBayes")\
                .set("spark.executor.memory", "6g")\
                .set("spark.driver.memory", "6g")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName("spark play").getOrCreate()

In [6]:
train = spark.read.csv("data/features/features/part-00000-83ff7fec-ad4a-4749-85b4-e8c10d87b30b-c000.csv", header=True, mode="DROPMALFORMED", inferSchema='true', encoding="utf-8").persist()
print train.count()
train.show()

186716
+--------------------+-----------------------+---------------------+-------------------+---------+---------+-------------+--------+--------+---------+---------+---------+---------+---------+---------+---------+-------------+-------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|           device_id|events_per_device_count|apps_per_device_count| apps_per_event_avg|lat_count|lng_count|lat_lng_count|min_hour|max_hour|mon_count|tue_count|wed_count|thu_count|fri_count|sat_count|sun_count|weekend_count|weekday_count|am_count|pm_count|h0_count|h1_count|h2_count|h3_count|h4_count|h5_count|h6_count|h7_count|h8_count|h9_count|h10_count|h11_count|h12_count|h13_count|h14_count|h15_count|h16_count|h17_count|h18_count|h19_count|h20_count|h21_count|h22_count|h23_count|geo_found|

In [7]:
# test = spark.read.csv("data/features/test/part-00000-3805f11b-4356-4463-8f94-e7491f0f1d91-c000.csv", header=True, mode="DROPMALFORMED", inferSchema='true', encoding="utf-8").persist()
# print test.count()
# test.show()

# Vector Assembler
- 

In [15]:
assembler = VectorAssembler(
                inputCols=[
                    "device_id",
                    "events_per_device_count",
                    "apps_per_device_count",
                    "apps_per_event_avg",
                    "lat_count",
                    "lng_count",
                    "lat_lng_count",
                    "min_hour",
                    "max_hour",
                    "mon_count",
                    "tue_count",
                    "wed_count",
                    "thu_count",
                    "fri_count",
                    "sat_count",
                    "sun_count",
                    "weekend_count",
                    "weekday_count",
                    "am_count",
                    "pm_count",
                    "h0_count",
                    "h1_count",
                    "h2_count",
                    "h3_count",
                    "h4_count",
                    "h5_count",
                    "h6_count",
                    "h7_count",
                    "h8_count",
                    "h9_count",
                    "h10_count",
                    "h11_count",
                    "h12_count",
                    "h13_count",
                    "h14_count",
                    "h15_count",
                    "h16_count",
                    "h17_count",
                    "h18_count",
                    "h19_count",
                    "h20_count",
                    "h21_count",
                    "h22_count",
                    "h23_count"
                ],
                outputCol="features"
            )

# naive-bayes
- https://spark.apache.org/docs/2.2.0/ml-classification-regression.html#naive-bayes

In [16]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")\
        .setFeaturesCol("features")\
        .setLabelCol("geo_found")\
        .setPredictionCol("prediction")\
        .setProbabilityCol("probability")\
        .setRawPredictionCol("confidence")

# Pipeline
- https://spark.apache.org/docs/2.2.0/ml-pipeline.html#pipeline

In [17]:
pipeline = Pipeline(stages=[assembler, nb])

In [18]:
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# With 3 values for hashingTF.numFeatures and 2 values for lr.regParam,
# this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.
    
params = ParamGridBuilder()\
            .addGrid(nb.modelType, ["multinomial"])\
            .addGrid(nb.smoothing, [smoothing])\
            .build()    

In [19]:
# Iterate and choose the best fit model  
cross_validator = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=params,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=5)  # use 3+ folds in practice         
                
cross_validator_model = cross_validator.fit(train)

Py4JJavaError: An error occurred while calling o163.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 15, localhost, executor driver): java.lang.IllegalArgumentException: requirement failed: Naive Bayes requires nonnegative feature values but found (44,[0,1,2,3,4,5,6,8,12,13,17,18,19,20,41,43],[-8.1623343132017541E18,95.0,47.0,0.49473684210526314,624.0,624.0,624.0,23.0,311.0,313.0,624.0,20.0,604.0,20.0,293.0,311.0]).
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.ml.classification.NaiveBayes$.requireNonnegativeValues(NaiveBayes.scala:233)
	at org.apache.spark.ml.classification.NaiveBayes$$anonfun$4.apply(NaiveBayes.scala:141)
	at org.apache.spark.ml.classification.NaiveBayes$$anonfun$4.apply(NaiveBayes.scala:141)
	at org.apache.spark.ml.classification.NaiveBayes$$anonfun$7.apply(NaiveBayes.scala:166)
	at org.apache.spark.ml.classification.NaiveBayes$$anon

In [ ]:
# Get and Write Output

In [ ]:
output = crossValidatorModel.transform(test)\
                        .select("device_index","probability")

In [ ]:
print output.show()

In [ ]:
output.repartition(1).write.option("header", True).csv("data/output/nb")